In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy.optimize import minimize
import pyreadr
import os
import time
from scipy.stats import norm
import copy
import warnings
import utils, regression_fit
import pickle
from importlib import reload
reload(utils)
reload(regression_fit)
from utils import *
from regression_fit import *

home = os.getcwd()

## Dataset

In [32]:
GroupA = pyreadr.read_r(home + '\\Other_data\\GroupA.rda')
GroupA_df = pd.DataFrame(list(GroupA.values())[0])

GroupB = pyreadr.read_r(home + '\\Other_data\\GroupB.rda')
GroupB_df = pd.DataFrame(list(GroupB.values())[0])

GroupC = pyreadr.read_r(home + '\\Other_data\\GroupC.rda')
GroupC_df = pd.DataFrame(list(GroupC.values())[0])

GroupD = pyreadr.read_r(home + '\\Other_data\\GroupD.rda')
GroupD_df = pd.DataFrame(list(GroupD.values())[0])

GroupE = pyreadr.read_r(home + '\\Other_data\\GroupE.rda')
GroupE_df = pd.DataFrame(list(GroupE.values())[0])

GroupF = pyreadr.read_r(home + '\\Other_data\\GroupF.rda')
GroupF_df = pd.DataFrame(list(GroupF.values())[0])

GroupG = pyreadr.read_r(home + '\\Other_data\\GroupG.rda')
GroupG_df = pd.DataFrame(list(GroupG.values())[0])

GroupH = pyreadr.read_r(home + '\\Other_data\\GroupH.rda')
GroupH_df = pd.DataFrame(list(GroupH.values())[0])

GroupJ = pyreadr.read_r(home + '\\Other_data\\GroupJ.rda')
GroupJ_df = pd.DataFrame(list(GroupJ.values())[0])

GroupH = pyreadr.read_r(home + '\\Other_data\\GroupH.rda')
GroupH_df = pd.DataFrame(list(GroupH.values())[0])

GroupJ = pyreadr.read_r(home + '\\Other_data\\GroupJ.rda')
GroupJ_df = pd.DataFrame(list(GroupJ.values())[0])

GroupK = pyreadr.read_r(home + '\\Other_data\\GroupK.rda')
GroupK_df = pd.DataFrame(list(GroupK.values())[0])

GroupL = pyreadr.read_r(home + '\\Other_data\\GroupL.rda')
GroupL_df = pd.DataFrame(list(GroupL.values())[0])

GroupM = pyreadr.read_r(home + '\\Other_data\\GroupM.rda')
GroupM_df = pd.DataFrame(list(GroupM.values())[0])

GroupN = pyreadr.read_r(home + '\\Other_data\\GroupN.rda')
GroupN_df = pd.DataFrame(list(GroupN.values())[0])

GroupP = pyreadr.read_r(home + '\\Other_data\\GroupP.rda')
GroupP_df = pd.DataFrame(list(GroupP.values())[0])

In [33]:
regions_df = [GroupA_df, GroupB_df, GroupC_df, GroupD_df, GroupE_df, GroupF_df, GroupG_df, GroupH_df, GroupJ_df, GroupK_df,
             GroupL_df, GroupM_df, GroupN_df, GroupP_df]

idx_2020 = 17516
idx_2021 = 35104

In [34]:
regions = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P']
train_set = {}
test_set = {}

# The dataset of each region is split in train and test set
for (reg, df) in zip(regions, regions_df):
    # Train set
    train_set[reg] = df[720:87440] 
    #Test set
    test_set[reg] = df[87440:]
    
y_mat = np.zeros((len(test_set['A']),len(regions)))
for i,reg in enumerate(regions):
    y_mat[:,i] = test_set[reg]['node']

# Persistence benchmark

The prediction for each target net-load value is equal to the value of 2 or 7 days before

In [36]:
y_pers_benchmark = {}

for reg, df in zip(regions, regions_df):
    pb_reg = Persistence_Benchmark(df)
    y_pers_benchmark[reg] = pb_reg.predict(test_set[reg]['targetTime'],delay_days=2)

In [37]:
y_pers_benchmark_7 = {}

for reg, df in zip(regions, regions_df):
    pb_reg = Persistence_Benchmark(df)
    y_pers_benchmark_7[reg] = pb_reg.predict(test_set[reg]['targetTime'],delay_days=7)

In [38]:
yhat_persistence = np.zeros((len(test_set['A']),len(regions)))

for i, reg in enumerate(regions):
    yhat_persistence[:,i] = y_pers_benchmark[reg]
    
yhat_persistence_7 = np.zeros((len(test_set['A']),len(regions)))

for i, reg in enumerate(regions):
    yhat_persistence_7[:,i] = y_pers_benchmark_7[reg]

In [39]:
print("\nPersistence Benchmark 2 days")

print("\nnRMSE GAM in 2019: ", nRMSE(y_mat[:idx_2020], yhat_persistence[:idx_2020]))
print("nRMSE GAM in 2020: ", nRMSE(y_mat[idx_2020:idx_2021], yhat_persistence[idx_2020:idx_2021]))
print("nRMSE GAM in 2021: ", nRMSE(y_mat[idx_2021:], yhat_persistence[idx_2021:]))

print("\nnMAE GAM in 2019: ", nMAE(y_mat[:idx_2020], yhat_persistence[:idx_2020]))
print("nMAE GAM in 2020: ", nMAE(y_mat[idx_2020:idx_2021], yhat_persistence[idx_2020:idx_2021]))
print("nMAE GAM in 2021: ", nMAE(y_mat[idx_2021:], yhat_persistence[idx_2021:]))

print("\n\nPersistence Benchmark 7 days")

print("\nnRMSE GAM in 2019: ", nRMSE(y_mat[:idx_2020], yhat_persistence_7[:idx_2020]))
print("nRMSE GAM in 2020: ", nRMSE(y_mat[idx_2020:idx_2021], yhat_persistence_7[idx_2020:idx_2021]))
print("nRMSE GAM in 2021: ", nRMSE(y_mat[idx_2021:], yhat_persistence_7[idx_2021:]))

print("\nnMAE GAM in 2019: ", nMAE(y_mat[:idx_2020], yhat_persistence_7[:idx_2020]))
print("nMAE GAM in 2020: ", nMAE(y_mat[idx_2020:idx_2021], yhat_persistence_7[idx_2020:idx_2021]))
print("nMAE GAM in 2021: ", nMAE(y_mat[idx_2021:], yhat_persistence_7[idx_2021:]))


Persistence Benchmark 2 days

nRMSE GAM in 2019:  0.7657144432686483
nRMSE GAM in 2020:  0.7637086247264843
nRMSE GAM in 2021:  0.7364503505051879

nMAE GAM in 2019:  0.6853608197041662
nMAE GAM in 2020:  0.6695116843860172
nMAE GAM in 2021:  0.6679449917889729


Persistence Benchmark 7 days

nRMSE GAM in 2019:  0.6899391491352814
nRMSE GAM in 2020:  0.698154186183202
nRMSE GAM in 2021:  0.737153692013546

nMAE GAM in 2019:  0.5876394746673863
nMAE GAM in 2020:  0.5868525831500012
nMAE GAM in 2021:  0.6389679156783059


# GAM-Point model

Results obtained in R with the bam function of the mgcv package 

In [42]:
Y_GAM_R_bam = {}

GroupA_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupA_GAM_R_bam.rda')
y_GAM_R_A = pd.DataFrame(list(GroupA_R.values())[0])
Y_GAM_R_bam["A"] = np.array(y_GAM_R_A).flatten()

GroupB_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupB_GAM_R_bam.rda')
y_GAM_R_B = pd.DataFrame(list(GroupB_R.values())[0])
Y_GAM_R_bam["B"] = np.array(y_GAM_R_B).flatten()

GroupC_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupC_GAM_R_bam.rda')
y_GAM_R_C = pd.DataFrame(list(GroupC_R.values())[0])
Y_GAM_R_bam["C"] = np.array(y_GAM_R_C).flatten()

GroupD_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupD_GAM_R_bam.rda')
y_GAM_R_D = pd.DataFrame(list(GroupD_R.values())[0])
Y_GAM_R_bam["D"] = np.array(y_GAM_R_D).flatten()

GroupE_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupE_GAM_R_bam.rda')
y_GAM_R_E = pd.DataFrame(list(GroupE_R.values())[0])
Y_GAM_R_bam["E"] = np.array(y_GAM_R_E).flatten()

GroupF_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupF_GAM_R_bam.rda')
y_GAM_R_F = pd.DataFrame(list(GroupF_R.values())[0])
Y_GAM_R_bam["F"] = np.array(y_GAM_R_F).flatten()

GroupG_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupG_GAM_R_bam.rda')
y_GAM_R_G = pd.DataFrame(list(GroupG_R.values())[0])
Y_GAM_R_bam["G"] = np.array(y_GAM_R_G).flatten()

GroupH_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupH_GAM_R_bam.rda')
y_GAM_R_H = pd.DataFrame(list(GroupH_R.values())[0])
Y_GAM_R_bam["H"] = np.array(y_GAM_R_H).flatten()

GroupJ_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupJ_GAM_R_bam.rda')
y_GAM_R_J = pd.DataFrame(list(GroupJ_R.values())[0])
Y_GAM_R_bam["J"] = np.array(y_GAM_R_J).flatten()

GroupH_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupH_GAM_R_bam.rda')
y_GAM_R_H = pd.DataFrame(list(GroupH_R.values())[0])
Y_GAM_R_bam["H"] = np.array(y_GAM_R_H).flatten()

GroupJ_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupJ_GAM_R_bam.rda')
y_GAM_R_J = pd.DataFrame(list(GroupJ_R.values())[0])
Y_GAM_R_bam["J"] = np.array(y_GAM_R_J).flatten()

GroupK_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupK_GAM_R_bam.rda')
y_GAM_R_K = pd.DataFrame(list(GroupK_R.values())[0])
Y_GAM_R_bam["K"] = np.array(y_GAM_R_K).flatten()

GroupL_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupL_GAM_R_bam.rda')
y_GAM_R_L = pd.DataFrame(list(GroupL_R.values())[0])
Y_GAM_R_bam["L"] = np.array(y_GAM_R_L).flatten()

GroupM_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupM_GAM_R_bam.rda')
y_GAM_R_M = pd.DataFrame(list(GroupM_R.values())[0])
Y_GAM_R_bam["M"] = np.array(y_GAM_R_M).flatten()

GroupN_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupN_GAM_R_bam.rda')
y_GAM_R_N = pd.DataFrame(list(GroupN_R.values())[0])
Y_GAM_R_bam["N"] = np.array(y_GAM_R_N).flatten()

GroupP_R = pyreadr.read_r(home + '\\Other_data\\Results R\\GroupP_GAM_R_bam.rda')
y_GAM_R_P = pd.DataFrame(list(GroupP_R.values())[0])
Y_GAM_R_bam["P"] = np.array(y_GAM_R_P).flatten()

In [43]:
idx_2020 = 17516
idx_2021 = 35104

yhat_GAM_mat_R_bam = np.zeros((len(test_set['A']),len(regions)))

for i, reg in enumerate(regions):
    yhat_GAM_mat_R_bam[:,i] = Y_GAM_R_bam[reg]

print("\nGAM-Point model")

print("\nnRMSE GAM in 2019: ", nRMSE(y_mat[:idx_2020], yhat_GAM_mat_R_bam[:idx_2020]))
print("nRMSE GAM in 2020: ", nRMSE(y_mat[idx_2020:idx_2021], yhat_GAM_mat_R_bam[idx_2020:idx_2021]))
print("nRMSE GAM in 2021: ", nRMSE(y_mat[idx_2021:], yhat_GAM_mat_R_bam[idx_2021:]))

print("\nnMAE GAM in 2019: ", nMAE(y_mat[:idx_2020], yhat_GAM_mat_R_bam[:idx_2020]))
print("nMAE GAM in 2020: ", nMAE(y_mat[idx_2020:idx_2021], yhat_GAM_mat_R_bam[idx_2020:idx_2021]))
print("nMAE GAM in 2021: ", nMAE(y_mat[idx_2021:], yhat_GAM_mat_R_bam[idx_2021:]))



GAM-Point model

nRMSE GAM in 2019:  0.3554482982247092
nRMSE GAM in 2020:  0.48519379380764477
nRMSE GAM in 2021:  0.6351325513177465

nMAE GAM in 2019:  0.32637142975291267
nMAE GAM in 2020:  0.4524598866590619
nMAE GAM in 2021:  0.6011450814429554


# Static Kalman GAM

Results obtained in R

In [46]:
Y_KF_static_delay_R = {}

GroupA_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupA_KF_static_delay.Rda')
Y_7d_R_A = pd.DataFrame(list(GroupA_R.values())[0])
Y_KF_static_delay_R["A"] = np.array(Y_7d_R_A).flatten()

GroupB_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupB_KF_static_delay.Rda')
Y_7d_R_B = pd.DataFrame(list(GroupB_R.values())[0])
Y_KF_static_delay_R["B"] = np.array(Y_7d_R_B).flatten()

GroupC_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupC_KF_static_delay.Rda')
Y_7d_R_C = pd.DataFrame(list(GroupC_R.values())[0])
Y_KF_static_delay_R["C"] = np.array(Y_7d_R_C).flatten()

GroupD_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupD_KF_static_delay.Rda')
Y_7d_R_D = pd.DataFrame(list(GroupD_R.values())[0])
Y_KF_static_delay_R["D"] = np.array(Y_7d_R_D).flatten()

GroupE_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupE_KF_static_delay.Rda')
Y_7d_R_E = pd.DataFrame(list(GroupE_R.values())[0])
Y_KF_static_delay_R["E"] = np.array(Y_7d_R_E).flatten()

GroupF_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupF_KF_static_delay.Rda')
Y_7d_R_F = pd.DataFrame(list(GroupF_R.values())[0])
Y_KF_static_delay_R["F"] = np.array(Y_7d_R_F).flatten()

GroupG_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupG_KF_static_delay.Rda')
Y_7d_R_G = pd.DataFrame(list(GroupG_R.values())[0])
Y_KF_static_delay_R["G"] = np.array(Y_7d_R_G).flatten()

GroupH_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupH_KF_static_delay.Rda')
Y_7d_R_H = pd.DataFrame(list(GroupH_R.values())[0])
Y_KF_static_delay_R["H"] = np.array(Y_7d_R_H).flatten()

GroupJ_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupJ_KF_static_delay.Rda')
Y_7d_R_J = pd.DataFrame(list(GroupJ_R.values())[0])
Y_KF_static_delay_R["J"] = np.array(Y_7d_R_J).flatten()

GroupH_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupH_KF_static_delay.Rda')
Y_7d_R_H = pd.DataFrame(list(GroupH_R.values())[0])
Y_KF_static_delay_R["H"] = np.array(Y_7d_R_H).flatten()

GroupJ_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupJ_KF_static_delay.Rda')
Y_7d_R_J = pd.DataFrame(list(GroupJ_R.values())[0])
Y_KF_static_delay_R["J"] = np.array(Y_7d_R_J).flatten()

GroupK_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupK_KF_static_delay.Rda')
Y_7d_R_K = pd.DataFrame(list(GroupK_R.values())[0])
Y_KF_static_delay_R["K"] = np.array(Y_7d_R_K).flatten()

GroupL_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupL_KF_static_delay.Rda')
Y_7d_R_L = pd.DataFrame(list(GroupL_R.values())[0])
Y_KF_static_delay_R["L"] = np.array(Y_7d_R_L).flatten()

GroupM_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupM_KF_static_delay.Rda')
Y_7d_R_M = pd.DataFrame(list(GroupM_R.values())[0])
Y_KF_static_delay_R["M"] = np.array(Y_7d_R_M).flatten()

GroupN_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupN_KF_static_delay.Rda')
Y_7d_R_N = pd.DataFrame(list(GroupN_R.values())[0])
Y_KF_static_delay_R["N"] = np.array(Y_7d_R_N).flatten()

GroupP_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupP_KF_static_delay.Rda')
Y_7d_R_P = pd.DataFrame(list(GroupP_R.values())[0])
Y_KF_static_delay_R["P"] = np.array(Y_7d_R_P).flatten()

In [47]:
idx_2020 = 17516
idx_2021 = 35104

yhat_KF_static_delay_mat_R = np.zeros((len(test_set['A']),len(regions)))

for i, reg in enumerate(regions):
    yhat_KF_static_delay_mat_R[:,i] = Y_KF_static_delay_R[reg]
    
print("\nStatic Kalman GAM")

print("\nnRMSE Static KF in 2019: ", nRMSE(y_mat[:idx_2020], yhat_KF_static_delay_mat_R[:idx_2020]))
print("nRMSE Static KF in 2020: ", nRMSE(y_mat[idx_2020:idx_2021], yhat_KF_static_delay_mat_R[idx_2020:idx_2021]))
print("nRMSE Static KF in 2021: ", nRMSE(y_mat[idx_2021:], yhat_KF_static_delay_mat_R[idx_2021:]))

print("\nnMAE Static KF in 2019: ", nMAE(y_mat[:idx_2020], yhat_KF_static_delay_mat_R[:idx_2020]))
print("nMAE Static KF in 2020: ", nMAE(y_mat[idx_2020:idx_2021], yhat_KF_static_delay_mat_R[idx_2020:idx_2021]))
print("nMAE Static KF in 2021: ", nMAE(y_mat[idx_2021:], yhat_KF_static_delay_mat_R[idx_2021:]))



Static Kalman GAM

nRMSE Static KF in 2019:  0.338203943863475
nRMSE Static KF in 2020:  0.3755293802401946
nRMSE Static KF in 2021:  0.3822969574009974

nMAE Static KF in 2019:  0.3078849363929946
nMAE Static KF in 2020:  0.3491162527377908
nMAE Static KF in 2021:  0.3705900767346633


# Dynamic Kalman GAM

### Hyperparameters selected with the Reduced Grid Search (I) method

Results obtained in R

In [50]:
Y_KF_dynamic_delay_R = {}

GroupA_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupA_KF_dynamic_delay.Rda')
Y_7d_R_A = pd.DataFrame(list(GroupA_R.values())[0])
Y_KF_dynamic_delay_R["A"] = np.array(Y_7d_R_A).flatten()

GroupB_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupB_KF_dynamic_delay.Rda')
Y_7d_R_B = pd.DataFrame(list(GroupB_R.values())[0])
Y_KF_dynamic_delay_R["B"] = np.array(Y_7d_R_B).flatten()

GroupC_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupC_KF_dynamic_delay.Rda')
Y_7d_R_C = pd.DataFrame(list(GroupC_R.values())[0])
Y_KF_dynamic_delay_R["C"] = np.array(Y_7d_R_C).flatten()

GroupD_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupD_KF_dynamic_delay.Rda')
Y_7d_R_D = pd.DataFrame(list(GroupD_R.values())[0])
Y_KF_dynamic_delay_R["D"] = np.array(Y_7d_R_D).flatten()

GroupE_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupE_KF_dynamic_delay.Rda')
Y_7d_R_E = pd.DataFrame(list(GroupE_R.values())[0])
Y_KF_dynamic_delay_R["E"] = np.array(Y_7d_R_E).flatten()

GroupF_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupF_KF_dynamic_delay.Rda')
Y_7d_R_F = pd.DataFrame(list(GroupF_R.values())[0])
Y_KF_dynamic_delay_R["F"] = np.array(Y_7d_R_F).flatten()

GroupG_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupG_KF_dynamic_delay.Rda')
Y_7d_R_G = pd.DataFrame(list(GroupG_R.values())[0])
Y_KF_dynamic_delay_R["G"] = np.array(Y_7d_R_G).flatten()

GroupH_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupH_KF_dynamic_delay.Rda')
Y_7d_R_H = pd.DataFrame(list(GroupH_R.values())[0])
Y_KF_dynamic_delay_R["H"] = np.array(Y_7d_R_H).flatten()

GroupJ_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupJ_KF_dynamic_delay.Rda')
Y_7d_R_J = pd.DataFrame(list(GroupJ_R.values())[0])
Y_KF_dynamic_delay_R["J"] = np.array(Y_7d_R_J).flatten()

GroupH_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupH_KF_dynamic_delay.Rda')
Y_7d_R_H = pd.DataFrame(list(GroupH_R.values())[0])
Y_KF_dynamic_delay_R["H"] = np.array(Y_7d_R_H).flatten()

GroupJ_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupJ_KF_dynamic_delay.Rda')
Y_7d_R_J = pd.DataFrame(list(GroupJ_R.values())[0])
Y_KF_dynamic_delay_R["J"] = np.array(Y_7d_R_J).flatten()

GroupK_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupK_KF_dynamic_delay.Rda')
Y_7d_R_K = pd.DataFrame(list(GroupK_R.values())[0])
Y_KF_dynamic_delay_R["K"] = np.array(Y_7d_R_K).flatten()

GroupL_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupL_KF_dynamic_delay.Rda')
Y_7d_R_L = pd.DataFrame(list(GroupL_R.values())[0])
Y_KF_dynamic_delay_R["L"] = np.array(Y_7d_R_L).flatten()

GroupM_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupM_KF_dynamic_delay.Rda')
Y_7d_R_M = pd.DataFrame(list(GroupM_R.values())[0])
Y_KF_dynamic_delay_R["M"] = np.array(Y_7d_R_M).flatten()

GroupN_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupN_KF_dynamic_delay.Rda')
Y_7d_R_N = pd.DataFrame(list(GroupN_R.values())[0])
Y_KF_dynamic_delay_R["N"] = np.array(Y_7d_R_N).flatten()

GroupP_R = pyreadr.read_r(home + '\\Other_data\\KF R\\GroupP_KF_dynamic_delay.Rda')
Y_7d_R_P = pd.DataFrame(list(GroupP_R.values())[0])
Y_KF_dynamic_delay_R["P"] = np.array(Y_7d_R_P).flatten()

In [51]:
yhat_KF_dynamic_delay_mat_R = np.zeros((len(test_set['A']),len(regions)))

for i, reg in enumerate(regions):
    yhat_KF_dynamic_delay_mat_R[:,i] = Y_KF_dynamic_delay_R[reg]

In [52]:
idx_2020 = 17516
idx_2021 = 35104

print("\nDynamic Kalman GAM")

print("\nnRMSE dynamic KF on bam with 48h delay in 2019: ", nRMSE(y_mat[:idx_2020], yhat_KF_dynamic_delay_mat_R[:idx_2020]))
print("nRMSE dynamic KF on bam with 48h delay in 2020: ", nRMSE(y_mat[idx_2020:idx_2021], yhat_KF_dynamic_delay_mat_R[idx_2020:idx_2021]))
print("nRMSE dynamic KF on bam with 48h delay in 2021: ", nRMSE(y_mat[idx_2021:], yhat_KF_dynamic_delay_mat_R[idx_2021:]))

print("\nnMAE dynamic KF on bam with 48h delay in 2019: ", nMAE(y_mat[:idx_2020], yhat_KF_dynamic_delay_mat_R[:idx_2020]))
print("nMAE dynamic KF on bam with 48h delay in 2020: ", nMAE(y_mat[idx_2020:idx_2021], yhat_KF_dynamic_delay_mat_R[idx_2020:idx_2021]))
print("nMAE dynamic KF on bam with 48h delay in 2021: ", nMAE(y_mat[idx_2021:], yhat_KF_dynamic_delay_mat_R[idx_2021:]))



Dynamic Kalman GAM

nRMSE dynamic KF on bam with 48h delay in 2019:  0.3299445064904641
nRMSE dynamic KF on bam with 48h delay in 2020:  0.3455332454592749
nRMSE dynamic KF on bam with 48h delay in 2021:  0.3453436009556959

nMAE dynamic KF on bam with 48h delay in 2019:  0.29794679708459926
nMAE dynamic KF on bam with 48h delay in 2020:  0.31987926982484644
nMAE dynamic KF on bam with 48h delay in 2021:  0.3252124793585592
